In [177]:
import csv
import numpy as np
import torch
import matplotlib.pyplot as plt
import torch.nn.functional as F

In [46]:
names = []
chars = set()
with open('./Dataset-Indian-Names-master/Indian_Names.csv','r') as f:
    lines = csv.reader(f)
    # lines = f.readlines()
    header = next(lines)
    for row in lines:
        name = row[1].lower()
        f=0
        for char in name:
            if not char>='a' and char <='z':
                # print(name)
                f=1
                break
            chars.add(char)
        if f==0:
            names.append(name)

In [37]:
chars = sorted(chars)

In [42]:
itos = {i+1:ch for i,ch in enumerate(chars)}
itos[0] = '.'

stoi = {ch:i for i,ch in itos.items()}

In [157]:
cnt_matrix = torch.zeros(len(itos), len(itos), dtype=torch.int32)
for name in names:
    newname = f'.{name}.'
    for i,j in zip(newname,newname[1:]):
        cnt_matrix[stoi[i]][stoi[j]] +=1

In [158]:
cnt_matrix

tensor([[   0,  586,  398,  140,  296,   18,  107,  228,  196,  100,  297,  378,
          125,  549,  361,   29,  377,    0,  548, 1217,  178,   58,  182,   17,
            0,   57,   25],
        [1552,  150,  233,   32,  226,    9,   67,  106,  223,  182,  289,  252,
          549,  658, 1402,    1,   51,    9,  932,  403,  382,   60,  163,   74,
           14,  142,  103],
        [  80,  260,   30,    1,    4,   50,    0,    0,  240,  168,    6,    0,
            6,    0,    5,   23,    1,    0,   30,    8,    0,   71,    0,    2,
            0,    8,    1],
        [   2,    7,    0,    6,    0,    6,    0,    0,  259,    1,    0,   12,
            0,    0,    0,    3,    0,    0,    0,    0,    1,    1,    0,    0,
            0,    4,    0],
        [ 191,  230,    2,    0,   73,  312,    0,    1,  211,  200,    0,    4,
            2,    5,    9,   20,    2,    0,  102,    4,    3,   65,    4,    5,
            0,    9,    0],
        [  94,    6,   19,    2,   34,  502,    7,

In [148]:
# plt.figure(figsize=(13,13))
# plt.imshow(cnt_matrix, cmap='Blues')
# for i in range(27):
#     for j in range(27):
#         chstr = itos[i] + itos[j]
#         plt.text(j, i, chstr, ha="center", va="bottom", color='gray')
#         plt.text(j, i, cnt_matrix[i, j].item(), ha="center", va="top", color='gray')
# plt.axis('off');

In [149]:
# cnt_matrix += 1
cnt_matrix = cnt_matrix + 1

In [159]:
probab_matrix = (cnt_matrix + 1)/torch.sum((cnt_matrix +1),1, keepdim=True)

In [333]:
g = torch.Generator().manual_seed(2147483647)

In [334]:
for _ in range(10):
    ch = 0
    str_ = ""
    while True:
        ch = torch.multinomial(probab_matrix[ch], replacement = True,num_samples=1, generator = g).item()
        if ch == 0:
            break
        str_= str_ + itos[ch]
    print(str_)
    

deeba
momakuraropritahampanimjanainjakadk
ka
da
samitabhjabhr
gotashishishanau
ka
teda
ka
hakamsade


In [162]:
log_likelihood = 0.0
cnt = 0

for name in ['tanmayy']:
    newname = f'.{name}.'
    for i,j in zip(newname,newname[1:]):
        log_likelihood += torch.log(probab_matrix[stoi[i]][stoi[j]])
        cnt += 1

print(f'{log_likelihood=}')
nll = -log_likelihood
print(f'{nll=}')
print(f'{nll/cnt}')

log_likelihood=tensor(-25.2468)
nll=tensor(25.2468)
3.155845880508423


## Creating Bigram

In [172]:
xs = []
ys = []

for name in names:
    newname = f'.{name}.'
    for i,j in zip(newname,newname[1:]):
        xs.append(stoi[i])
        ys.append(stoi[j])

xs = torch.tensor(xs)
ys = torch.tensor(ys)

In [165]:
# xs

In [186]:
# xs_oh = F.one_hot(xs)

In [368]:
# randomly initialize 27 neurons' weights. each neuron receives 27 inputs
g = torch.Generator().manual_seed(2147483647)
W = torch.randn((27, 27), generator=g,requires_grad=True)

In [369]:
# y_pred = xs_oh.float() @ w

In [378]:
for k in range(10):
    # forward pass
    xenc = F.one_hot(xs, num_classes=27).float() # input to the network: one-hot encoding
    logits = xenc @ W # predict log-counts
    counts = logits.exp() # counts, equivalent to N
    probs = counts / counts.sum(1, keepdims=True) # probabilities for next character
    # btw: the last 2 lines here are together called a 'softmax'
    nll = -probs[torch.arange(probs.shape[0]),ys].log().mean()  + 0.01*(W**2).mean() #also the loss, second term is regularization(pushing ws to 0)
    print(f"Loss = {nll.data}")
    
    #backward pass
    W.grad = None # set to zero the gradient
    nll.backward()
    W.data += -50*W.grad

Loss = 2.3934316635131836
Loss = 2.3931963443756104
Loss = 2.392967700958252
Loss = 2.39274525642395
Loss = 2.392529249191284
Loss = 2.3923189640045166
Loss = 2.3921151161193848
Loss = 2.391916036605835
Loss = 2.3917229175567627
Loss = 2.3915343284606934


In [379]:
# finally, sample from the 'neural net' model after training Ws 
g = torch.Generator().manual_seed(2147483647)

for i in range(10):
    str_ = ''
    ix = 0
    while True:
        # ----------
        # BEFORE:
        #p = probab_matrix[ix]
        # ----------
        # NOW:
        xenc = F.one_hot(torch.tensor([ix]), num_classes=27).float()
        logits = xenc @ W # predict log-counts
        counts = logits.exp() # counts, equivalent to N
        p = counts / counts.sum(1, keepdims=True) # probabilities for next character

        ix = torch.multinomial(p, replacement = True,num_samples=1, generator = g).item()
        if ix == 0:
            break
        str_= str_ + itos[ix]
    print(str_)

deeba
momakuraropritahamem
imjanainjakadk
ka
da
samitabhjabhr
gotashiszishanau
ka
teda
ka


In [380]:
# finally, sample from the 'neural net' model after training Ws 
g = torch.Generator().manual_seed(2147483647)

for i in range(10):
    str_ = ''
    ix = 0
    while True:
        # ----------
        # BEFORE:
        p = probab_matrix[ix]
        # ----------
        # NOW:
        # xenc = F.one_hot(torch.tensor([ix]), num_classes=27).float()
        # logits = xenc @ W # predict log-counts
        # counts = logits.exp() # counts, equivalent to N
        # p = counts / counts.sum(1, keepdims=True) # probabilities for next character

        ix = torch.multinomial(p, replacement = True,num_samples=1, generator = g).item()
        if ix == 0:
            break
        str_= str_ + itos[ix]
    print(str_)

deeba
momakuraropritahampanimjanainjakadk
ka
da
samitabhjabhr
gotashishishanau
ka
teda
ka
hakamsade
